# Importing Functions 

In [3]:
import argparse
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import time 
from torch.autograd import grad
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import random
import scipy.io
torch.backends.cudnn.deterministic = True
device = torch.device('cuda')
seed = 999
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Models

In [4]:
class Encoder_AMDA(nn.Module):
    def __init__(self):
        super(Encoder_AMDA, self).__init__()
        self.encoder = nn.Sequential(
            # first layer  4096*1-->  1017*8
            nn.Conv1d(1, 8, kernel_size=32,stride=2, padding=1),
#             nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(2),
            # second layer  1017*8-->  250*16
            nn.Conv1d(8, 16, kernel_size=16,stride=2, padding=1),
#             nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            # third layer  250*16-->  60*32
            nn.Conv1d(16, 32, kernel_size=8,stride=2,padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2), 
            # fourth layer 60*32--> 14*32
            nn.Conv1d(32, 32, kernel_size=8,stride=2,padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            # fifth layer 14*32--> 3*64
            nn.Conv1d(32, 64, kernel_size=3,stride=2,padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2))
         # flatenning wit fully connected layers
        
        self.fc1 = nn.Linear(64*3, 256)# optimal when 0 source domain

    def forward(self, input):
        conv_out = self.encoder(input)
#         conv_out=F.dropout(conv_out)# we didn't need it when source domain is zero condition
        feat = self.fc1(conv_out.view(conv_out.shape[0],-1))
        return feat
            
    """classifier model for AMDA."""
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc2 = nn.Linear(256, 10) #this with 0 as souce it give optimal results 
  

    def forward(self, feat):
        out = F.dropout(F.relu(feat), training=self.training)
        out = self.fc2(out)
        return out
    
class Discriminator(nn.Module):
    """Discriminator model for source domain."""

    def __init__(self, input_dims, hidden_dims, output_dims):
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_dims, hidden_dims),
            nn.ReLU(),
            nn.Linear(hidden_dims, hidden_dims),
            nn.ReLU(),
            nn.Linear(hidden_dims, output_dims),
            nn.LogSoftmax())

    def forward(self, input):
        
        out = self.layer(input)
        return out

# CWRU Data Loading 

In [5]:
# load 0
source_data=torch.load('./data/source_data_4096.pt')
source_labels_0=torch.load('./data/source_labels_4096.pt')
test_data_0=torch.load('./data/test_data_4096.pt')
test_labels_0=torch.load('./data/test_labels_4096.pt')
# load 1
target_data_a=torch.load('./data/target_data_a_4096.pt')
target_labels_a=torch.load('./data/target_labels_a_4096.pt')
test_a=torch.load('./data/test_data_a_4096.pt')
test_a_labels=torch.load('./data/test_labels_a_4096.pt')
# load 2
target_data_b=torch.load('./data/target_data_b_4096.pt')
target_labels_b=torch.load('./data/target_labels_b_4096.pt')
test_b=torch.load('./data/test_data_b_4096.pt')
test_b_labels=torch.load('./data/test_labels_b_4096.pt')
# load 3
target_data_c=torch.load('./data/target_data_c_4096.pt')
target_labels_c=torch.load('./data/target_labels_c_4096.pt')
test_c=torch.load('./data/test_data_c_4096.pt')
test_c_labels=torch.load('./data/test_labels_c_4096.pt')

### Choosing differnt source and target

In [6]:
# training 
source_domain=source_data.float()
source_labels=source_labels_0.long()
# testing on same distribution 
test_data=test.float()
test_labels=test_b_labels.long()

#Testing on differnt domain 
target_domain_0=target_data_a.float()
target_labels_0=target_labels_a.long()
target_domain_1=target_data_b.float()
target_labels_1=target_labels_b.long()
target_domain_2=target_data_c.float()
target_labels_2=target_labels_c.long()


sample_length=source_data.size(1)
num_samples=source_data.size(0)
num_test_samples=test_data.size(0)
print(num_test_samples)
print(sample_length)

3200
4096


# Parameters

In [16]:
"""Params for AMDA."""
d_input_dims = 256 
d_hidden_dims = 256 
d_output_dims = 2
# params for training network
num_epochs_pre = 100
log_step_pre = 10
eval_step_pre = 20
save_step_pre = 100
num_epochs = 150
log_step = 10
save_step = 100
# params for optimizing models
d_learning_rate = 1e-4
c_learning_rate = 1e-4
beta1 = 0.5
beta2 = 0.9
bs=20

# Pre-Training 

In [44]:
def train_src(encoder, classifier):
    """Train classifier for source domain."""
    ####################
    # 1. setup network #
    ####################
    # set train state for Dropout and BN layers
    src_encoder.train()
    classifier.train()

    # setup criterion and optimizer
    optimizer = optim.Adam(
        list(encoder.parameters()) + list(classifier.parameters()),
        lr=c_learning_rate,
        betas=(beta1, beta2))
    criterion = nn.CrossEntropyLoss()

    ####################
    # 2. train network #
    ####################

    for epoch in range(num_epochs_pre):
        running_loss=0
        running_accuracy=0
        num_batches=0
        for step in range(0,num_samples,bs):
            # training on target domain_a as a source
            minibatch_data =  source_domain[step:step+bs].unsqueeze(dim=1)
            minibatch_label=  source_labels[step:step+bs].squeeze()
            minibatch_data=minibatch_data.to(device)
            minibatch_label=minibatch_label.to(device)
    
            # zero gradients for optimizer
            optimizer.zero_grad()
            
            # compute loss for critic
            preds =classifier (encoder(minibatch_data))
            loss = criterion(preds, minibatch_label)
            
            # optimize source classifier
            loss.backward()
            optimizer.step()
            running_loss += loss.detach().item()
        
            running_accuracy += (preds.max(1)[1] == minibatch_label).float().mean().item()
            num_batches+=1
        # print epoch info
        if ((epoch) % log_step_pre == 0):
            print("Epoch [{}/{}] : loss={} train_accuracy={}"
                  .format(epoch,
                          num_epochs_pre,
                          running_loss/num_batches,
                        running_accuracy*100/num_batches ))

        # eval model on test set
        if ((epoch + 1) % eval_step_pre == 0):
            eval_src(encoder, classifier)

        # save model parameters
        if ((epoch + 1) % save_step_pre == 0):
            torch.save(encoder.state_dict(), 'M_T_src_encoder-{}.pt'.format(epoch + 1))
            torch.save(classifier.state_dict(), 'M_T_src-classifier-{}.pt'.format(epoch + 1))

    # # save final model
    torch.save(encoder.state_dict(), 'M_T_src-encoder-final.pt')
    torch.save(classifier.state_dict(), 'M_T_src-classifier-final.pt')

    return encoder, classifier
def eval_src(encoder, classifier):
    """Evaluate classifier for source domain."""
    # set eval state for Dropout and BN layers
    encoder.eval()
    classifier.eval()
    # init loss and accuracy
    loss = 0
    acc = 0
    num_batches=0
    # set loss function
    criterion = nn.CrossEntropyLoss()
    for i in range(0,num_test_samples,bs):
        minibatch_data =  test_data[i:i+bs].unsqueeze(dim=1)
        minibatch_label= test_labels[i:i+bs].squeeze()
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # forward pass
        scores=classifier(encoder(minibatch_data))
        
        # calculate accuracy
                               
        acc += (scores.max(1)[1] == minibatch_label).float().mean().item()
        loss += criterion(scores, minibatch_label)
        num_batches+=1
    mean_accuracy = acc / num_batches
#     acc_plt.append(mean_accuracy)
    mean_loss = loss / num_batches
#     loss_plt.append(mean_loss)
    print("Avg Loss = {}, Avg Accuracy = {:2%}".format(mean_loss, mean_accuracy))


# Adapting 

In [14]:
def train_tgt(src_encoder, tgt_encoder, critic):
    """Train encoder for target domain."""
    ####################
    # 1. setup network #
    ####################

    # set train state for Dropout and BN layers
    tgt_encoder.train()
    critic.train()

    # setup criterion and optimizer
    
    criterion = nn.CrossEntropyLoss()
    optimizer_tgt = optim.Adam(tgt_encoder.parameters(),
                               lr=c_learning_rate,
                               betas=(beta1, beta2))
    optimizer_critic = optim.Adam(critic.parameters(),
                                  lr=d_learning_rate,
                                  betas=(beta1, beta2))
    len_data_loader =num_samples

    ####################
    # 2. train network #
    ####################

    for epoch in range(num_epochs):
        # initilize loss
        run_critic_loss=0
        acc=0
        run_tgt_loss=0
        num_batches=0
        for i in range(0,num_samples,bs):
            ###########################
            # 2.1 train discriminator #
            ###########################

            # make images variable
            sample_src= source_domain[i:i+bs].float().unsqueeze(dim=1).to(device)
            sample_tgt_1=target_domain_0[i:i+bs].float().unsqueeze(dim=1).to(device)
            sample_tgt_2=target_domain_1[i:i+bs].float().unsqueeze(dim=1).to(device)
            sample_tgt_3=target_domain_2[i:i+bs].float().unsqueeze(dim=1).to(device)
            # zero gradients for optimizer
            optimizer_critic.zero_grad()

            # extract and concat features
            feat_src = src_encoder(sample_src)
            feat_tgt_1 = tgt_encoder(sample_tgt_1)
            feat_tgt_2 = tgt_encoder(sample_tgt_2)
            feat_tgt_3 = tgt_encoder(sample_tgt_3)
            feat_tgt= torch.cat((feat_tgt_1,feat_tgt_2),0)
#             feat_tgt= torch.cat((feat_tgt_1,feat_tgt_2,feat_tgt_3),0)

            feat_concat = torch.cat((feat_src, feat_tgt), 0)
            # predict on discriminator
            pred_concat = critic(feat_concat.detach()).to(device)

            # prepare real and fake label
            label_src =Variable(torch.ones(feat_src.size(0)).long())
            label_tgt = Variable(torch.zeros(feat_tgt.size(0)).long())
            label_concat = torch.cat((label_src, label_tgt), 0).to(device)

            # compute loss for critic
            loss_critic = criterion(pred_concat, label_concat)
            loss_critic.backward()

            # optimize critic
            optimizer_critic.step()

            pred_cls = torch.squeeze(pred_concat.max(1)[1])
            acc += (pred_cls == label_concat).float().mean()
            run_critic_loss+=  loss_critic.detach().item()
            ############################
            # 2.2 train target encoder #
            ############################
            optimizer_tgt.zero_grad() # edited here becareful 
            for i in range (2):
                # extract and target features
                feat_tgt_1 = tgt_encoder(sample_tgt_1)
                feat_tgt_2 = tgt_encoder(sample_tgt_2)
                feat_tgt_3 = tgt_encoder(sample_tgt_3)
                feat_tgt= torch.cat((feat_tgt_1,feat_tgt_2,feat_tgt_3),0)

                # predict on discriminator
                pred_tgt = critic(feat_tgt).to(device)

                # prepare fake labels to enforce the feature extractor to confuse the critic 
                label_tgt = Variable(torch.ones(feat_tgt.size(0)).long()).to(device)

                # compute loss for target encoder
                loss_tgt = criterion(pred_tgt, label_tgt)
                loss_tgt.backward()

                # optimize target encoder
                optimizer_tgt.step()

                run_tgt_loss+=loss_tgt.detach().item()
            num_batches+=1
        #######################
        # 2.3 print epoch info #
        #######################
        if ((epoch) % log_step == 0):
            print("Epoch [{}/{}] :"
                  "discriminator_loss={:.5f} target_loss={:.5f} discriminator_acc={:.5f}"
                  .format(epoch,
                         num_epochs,
                          run_critic_loss/num_batches,
                          run_tgt_loss/(num_batches*5),
                          acc.data[0]/num_batches))
            print("=== Evaluating classifier for encoded target domain ===")
            print(">>> source only <<<")
            eval_tgt(src_encoder, src_classifier)
            print(">>> domain adaption <<<")
            eval_tgt(tgt_encoder, src_classifier)


        #############################
        # 2.4 save model parameters #
        #############################
        if ((epoch + 1) % save_step == 0):
            torch.save(critic.state_dict(),
            'M_T_ADDA-critic-{}.pt'.format(epoch + 1))
            torch.save(tgt_encoder.state_dict(),
                'M_T_ADDA-target-encoder-{}.pt'.format(epoch + 1))

    torch.save(critic.state_dict(), 'M_T_ADDA-critic-final.pt')
    torch.save(tgt_encoder.state_dict(),'M_T_ADDA-target-encoder-final.pt')
    return tgt_encoder



def eval_tgt(encoder, classifier):
    """Evaluation for target encoder by source classifier on target dataset."""
    # set eval state for Dropout and BN layers
    encoder.eval()
    classifier.eval()

    # init loss and accuracy
    loss = 0
    acc = 0
    num_batches=0
    # set loss function
    
    criterion = nn.CrossEntropyLoss()
    x=[]
    y=[]
    # evaluate network
    for i in range(0,num_samples,bs):
        minibatch_data =  target_domain[i:i+bs].unsqueeze(dim=1)
        minibatch_label= target_labels[i:i+bs].squeeze()
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
#         inputs = (minibatch_data - src_mean)/src_std  
#         inputs=inputs.to(device)
        preds = classifier(encoder(minibatch_data))
        loss += criterion(preds, minibatch_label.squeeze()).data[0]
        x.append(preds.max(1)[1])
        y.append(minibatch_label)
        pred_cls = preds.data.max(1)[1]
#         acc += pred_cls.eq(minibatch_label.data).cpu().sum().float()
        acc +=(pred_cls == minibatch_label).float().mean()
        num_batches+=1
    loss /= num_batches
    acc /= num_batches
    
    print("Avg Loss = {}, Avg Accuracy = {}".format(loss, acc*100,'%'))
    return(x,y)

#  Main Code  

In [12]:
# load models
src_encoder = Encoder_AMDA().to(device)

src_classifier= Classifier().to(device)

tgt_encoder = Encoder_AMDA().to(device)

critic = Discriminator(input_dims=d_input_dims,
                                  hidden_dims=d_hidden_dims,
                                  output_dims=d_output_dims).to(device)

# train source model

print("=== Training classifier for source domain ===")
print(">>> Source Encoder <<<")
print(src_encoder)
print(">>> Source Classifier <<<")
print(src_classifier)

src_encoder, src_classifier = train_src(
        src_encoder, src_classifier)

# # eval source model
print("=== Evaluating classifier for source domain ===")
eval_src(src_encoder, src_classifier)

# # train target encoder by GAN
print("=== Training encoder for target domain ===")
print(">>> Target Encoder <<<")
print(tgt_encoder)
print(">>> Critic <<<")
print(critic)

# # init weights of target encoder with those of source encoder
tgt_encoder.load_state_dict(src_encoder.state_dict())
tgt_encoder = train_tgt(src_encoder, tgt_encoder, critic)

# # eval target encoder on test set of target dataset
print("=== Evaluating classifier for encoded target domain ===")
print(">>> source only <<<")
eval_tgt(src_encoder, src_classifier)
print(">>> domain adaption <<<")
eval_tgt(tgt_encoder, src_classifier)


=== Training classifier for source domain ===
>>> Source Encoder <<<
Encoder_ADDA(
  (encoder): Sequential(
    (0): Conv1d(1, 8, kernel_size=(32,), stride=(2,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(8, 16, kernel_size=(16,), stride=(2,), padding=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(16, 32, kernel_size=(8,), stride=(2,), padding=(1,))
    (7): ReLU()
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv1d(32, 32, kernel_size=(8,), stride=(2,), padding=(1,))
    (10): ReLU()
    (11): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv1d(32, 64, kernel_size=(3,), stride=(2,), padding=(1,))
    (13): ReLU()
    (14): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=192, out_fe

/home/mohamedragab1992/miniconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/home/mohamedragab1992/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:115: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/mohamedragab1992/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:161: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch [0/150] :discriminator_loss=0.99515 target_loss=0.32004 discriminator_acc=0.27375
=== Evaluating classifier for encoded target domain ===
>>> source only <<<
Avg Loss = 2.801046133041382, Avg Accuracy = 70.37500762939453
>>> domain adaption <<<
Avg Loss = 0.46660828590393066, Avg Accuracy = 77.9375
Epoch [10/150] :discriminator_loss=0.63907 target_loss=0.41618 discriminator_acc=0.65938
=== Evaluating classifier for encoded target domain ===
>>> source only <<<
Avg Loss = 2.801046133041382, Avg Accuracy = 70.37500762939453
>>> domain adaption <<<
Avg Loss = 5.421848297119141, Avg Accuracy = 66.75000762939453
Epoch [20/150] :discriminator_loss=0.64844 target_loss=0.46338 discriminator_acc=0.65521
=== Evaluating classifier for encoded target domain ===
>>> source only <<<
Avg Loss = 2.801046133041382, Avg Accuracy = 70.37500762939453
>>> domain adaption <<<
Avg Loss = 6.832834720611572, Avg Accuracy = 66.75000762939453
Epoch [30/150] :discriminator_loss=0.64583 target_loss=0.46220 d

KeyboardInterrupt: 

### Confusion Matrix, Precision and Recall


In [22]:
target_domain=target_data_a.float()
target_labels=target_labels_a.long()
target_domain=target_data_b.float()
target_labels=target_labels_b.long()
target_domain=target_data_c.float()
target_labels=target_labels_c.long()
# target_domain=source_data.float()
# target_labels=source_labels_0.long()

eval_tgt(src_encoder, src_classifier)
x,y=eval_tgt(tgt_encoder, src_classifier)

from sklearn.metrics import confusion_matrix
import pandas as pd

pred = torch.LongTensor(2816, 5).cuda()
torch.cat(x, out=pred)

actual = torch.LongTensor(2816, 5).cuda()
torch.cat(y, out=actual)

pred=pred.cpu()
actual=actual.cpu()
con= confusion_matrix(actual.numpy(),pred.numpy(),labels=[0,1,2,3,4,5,6,7,8,9])
recall = np.diag(con) / np.sum(con, axis = 1)
precision = np.diag(con) / np.sum(con, axis = 0)
avg_recall=np.mean(recall)*100
avg_precision=np.mean(precision)*100
f_measure= 2*(avg_recall*avg_precision)/(avg_precision+avg_recall)
con = con.astype('float') / con.sum(axis=1)[:, np.newaxis]
conf = pd.DataFrame(data=con)
conf.columns.name = 'Predicted label'
conf.index.name = 'Actual label'
print('precision:', avg_precision, 'recall:' , avg_recall, 'f-measure',f_measure)
conf

/home/mohamedragab1992/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:162: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Avg Loss = 2.312741994857788, Avg Accuracy = 9.979167938232422
Avg Loss = 0.07230367511510849, Avg Accuracy = 99.52081298828125
precision: 99.53821244782574 recall: 99.51991979131854 f-measure 99.52906527906067


Predicted label         0    1    2    3    4         5         6    7  \
Actual label                                                             
0                1.000000  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0   
1                0.000000  1.0  0.0  0.0  0.0  0.000000  0.000000  0.0   
2                0.000000  0.0  1.0  0.0  0.0  0.000000  0.000000  0.0   
3                0.000000  0.0  0.0  1.0  0.0  0.000000  0.000000  0.0   
4                0.000000  0.0  0.0  0.0  1.0  0.000000  0.000000  0.0   
5                0.002079  0.0  0.0  0.0  0.0  0.997921  0.000000  0.0   
6                0.000000  0.0  0.0  0.0  0.0  0.029228  0.954071  0.0   
7                0.000000  0.0  0.0  0.0  0.0  0.000000  0.000000  1.0   
8                0.000000  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0   
9                0.000000  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0   

Predicted label         8    9  
Actual label                    
0                0.000000  0.0  
1                0.000000  0.0  
2                0.000000  0.0  
3                0.000000  0.0  
4                0.000000  0.0  
5                0.000000  0.0  
6                0.016701  0.0  
7                0.000000  0.0  
8                1.000000  0.0  
9                0.000000  1.0